In [2]:
import cv2
import torch
from torch import nn
import datasets
import numpy as np
import matplotlib.pyplot as pl
import models

In [5]:

dataset = datasets.TDSC(path_to_dataset="data/tdsc")
modle = models.UNet2D(in_channels = 1 ,out_channels = 1)

In [ ]:
for sample in data